# Benchmarking Test

## Switching to GPU

In [1]:
import torch

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Define CNN Models

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

def load_resnet50(num_classes):
    model =  models.resnet50(weights='ResNet50_Weights.DEFAULT')
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    return model

def load_efficientnet_b0(num_classes):
    model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v2(num_classes):
    model = models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v3_large(num_classes):
    model = models.mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.DEFAULT')
    num_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v3_small(num_classes):
    model = models.mobilenet_v3_small(weights='MobileNet_V3_Small_Weights.DEFAULT')
    num_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(num_features, num_classes)
    return model

## Load Benchmarking Test Dataset

In [22]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_test_dataloader(data_dir, batch_size, num_workers=4):
    # Define transformations for the test dataset
    data_transform = transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Create the test dataset
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Benchmarking'), transform=data_transform)

    # Create the test dataloader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return test_loader

data_dir = 'Dataset/'
batch_size = 32
test_loader = create_test_dataloader(data_dir, batch_size)

In [23]:
from torch import nn
from torchvision import models
from tqdm import tqdm

# Function to load the model
def load_model(model_class, checkpoint_path, num_classes):
    model = model_class(num_classes=num_classes)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()
    return model

# Function to test the model
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1) 
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = test_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f'Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%')

In [24]:
# List of model loading functions
model_functions = [
    load_resnet50,
    load_mobilenet_v3_large,
    load_mobilenet_v3_large,
    load_efficientnet_b0,
    load_mobilenet_v2,
    load_mobilenet_v3_small,
]

checkpoint_paths = [
    'Checkpoints/resnet50/model_epoch_9.pth',
    'Checkpoints/mobilenet_v3_large/model_epoch_10.pth',
    'Checkpoints/mobilenet_v3_large_20ep/model_epoch_11.pth',
    'Checkpoints/efficientnet_b0/model_epoch_14.pth',
    'Checkpoints/mobilenet_v2/model_epoch_7.pth',
    'Checkpoints/mobilenet_v3_small/model_epoch_6.pth',
]

## Operate Benchmarking Test

In [25]:
num_classes = 3
loss_fn = nn.CrossEntropyLoss()

for get_model, checkpoint_path in zip(model_functions, checkpoint_paths):
    # Load the model using the function
    loaded_model = load_model(get_model, checkpoint_path, num_classes)
    loaded_model = loaded_model.to(device)
    # Evaluate the loaded model on the test set
    print(checkpoint_path)
    evaluate_model(loaded_model, test_loader, loss_fn)

Checkpoints/resnet50/model_epoch_9.pth


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.01it/s]


Loss: 0.0786, Accuracy: 96.83%
Checkpoints/mobilenet_v3_large/model_epoch_10.pth


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.29it/s]


Loss: 0.0270, Accuracy: 98.83%
Checkpoints/mobilenet_v3_large_20ep/model_epoch_11.pth


...

Loss: 0.0078, Accuracy: 100.00%
Checkpoints/mobilenet_v2/model_epoch_7.pth


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.19it/s]


Loss: 0.0506, Accuracy: 98.17%
Checkpoints/mobilenet_v3_small/model_epoch_6.pth


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.46it/s]

Loss: 0.0724, Accuracy: 97.67%
